In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_2D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            if a(j*dt) == 0:
                continue
            
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_2D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0] - prms[1]*t
    a2 = lambda t: (a(t))**2
    da2 = lambda t: -2*prms[1] * a(t)
    
    ndt = prms[2]
    mu = np.array([prms[3], prms[4]])
        
    T_max = min(max(RT), prms[0]/prms[1])
    fpt = ie_bessel_fpt(a2, da2, mu.shape[0], 0.000001, 
                        dt=0.02, T_max=T_max)
        
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.0001 and rt - ndt < T_max:
            mu_dot_x0 = mu[0]*np.cos(theta)
            mu_dot_x1 = mu[1]*np.sin(theta)
            term1 = a(rt - ndt) * (mu_dot_x0 + mu_dot_x1)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt - ndt)

            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'b0_true': [],
               'b0_estimate': [],
               'lambda_true': [],
               'lambda_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_b0 = 2
max_b0 = 5

min_lambda = .1
max_lambda = 2

min_ndt = 0.1
max_ndt = 1

min_mu = -3
max_mu = 3

In [6]:
for n in tqdm(range(50)):
    b0 = np.random.uniform(min_b0, max_b0)
    lamb = np.random.uniform(min_lambda, max_lambda)
    a = lambda t: b0 - lamb*t
    
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    RT = []
    Theta = []
    
    for i in range(750):
        rt, theta = simulate_HSDM_2D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_2D_likelihood,
                                     args=(RT, Theta),
                                     bounds=[(min_b0, max_b0), (min_lambda, max_lambda), 
                                             (min_ndt, max_ndt), (min_mu, max_mu), (min_mu, max_mu)])
    
    min_ans = minimize(HSDM_2D_likelihood,
                       args=(RT, Theta),
                       method='nelder-mead',
                       x0=min_ans.x,
                       bounds=[(min_b0, max_b0), (min_lambda, max_lambda),
                               (min_ndt, max_ndt), (min_mu, max_mu), (min_mu, max_mu)])
    
    if min_ans.success:
        recovery_df['b0_true'].append(b0)
        recovery_df['lambda_true'].append(lamb)
        recovery_df['ndt_true'].append(ndt)
        recovery_df['mu1_true'].append(mu[0])
        recovery_df['mu2_true'].append(mu[1])

        recovery_df['b0_estimate'].append(min_ans.x[0])
        recovery_df['lambda_estimate'].append(min_ans.x[1])
        recovery_df['ndt_estimate'].append(min_ans.x[2])
        recovery_df['mu1_estimate'].append(min_ans.x[3])
        recovery_df['mu2_estimate'].append(min_ans.x[4])

100%|██████████████████████████████| 50/50 [4:31:35<00:00, 325.91s/it]


In [7]:
recovery_df = pd.DataFrame(recovery_df)
recovery_df.corr()

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
b0_true,1.000000,0.938728,-0.065787,-0.101559,0.102472,0.131016,0.137639,0.136232,-0.038603,-0.041526
b0_estimate,0.938728,1.000000,-0.079828,-0.024012,0.128988,0.092006,0.114114,0.114319,-0.061375,-0.063330
lambda_true,-0.065787,-0.079828,1.000000,0.954832,0.120195,0.122964,-0.142741,-0.145088,0.012637,0.010221
lambda_estimate,-0.101559,-0.024012,0.954832,1.000000,0.104562,0.064619,-0.202683,-0.202498,-0.002035,-0.004256
ndt_true,0.102472,0.128988,0.120195,0.104562,1.000000,0.977862,0.256011,0.252614,-0.110932,-0.106065
ndt_estimate,0.131016,0.092006,0.122964,0.064619,0.977862,1.000000,0.255759,0.252234,-0.090801,-0.086137
mu1_true,0.137639,0.114114,-0.142741,-0.202683,0.256011,0.255759,1.000000,0.999315,-0.119886,-0.118789
mu1_estimate,0.136232,0.114319,-0.145088,-0.202498,0.252614,0.252234,0.999315,1.000000,-0.122246,-0.121086
mu2_true,-0.038603,-0.061375,0.012637,-0.002035,-0.110932,-0.090801,-0.119886,-0.122246,1.000000,0.999361
mu2_estimate,-0.041526,-0.063330,0.010221,-0.004256,-0.106065,-0.086137,-0.118789,-0.121086,0.999361,1.000000


In [8]:
file_name = '_Recovery_data/IE_2d_recovery_linear_750_02.csv'
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
recovery_df

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,3.165785,3.357523,1.688990,1.955847,0.446924,0.439607,-2.874202,-2.799968,2.145640,2.144930
1,2.777051,2.574620,1.988227,1.750796,0.455396,0.491151,2.647520,2.713038,-1.327568,-1.396868
2,2.248839,2.185761,0.151527,0.115028,0.258798,0.294824,-0.247580,-0.246707,-1.450552,-1.398324
3,3.061472,2.877842,0.487501,0.377021,0.868788,0.921286,1.103401,1.127458,2.192582,2.175995
4,2.876503,2.557261,0.901173,0.716927,0.798065,0.865479,-0.218974,-0.263351,-1.556092,-1.590280
...,...,...,...,...,...,...,...,...,...,...
295,2.604365,2.999633,0.564008,0.828393,0.588630,0.522099,-2.673529,-2.682836,-0.617580,-0.637058
296,4.499336,4.377790,0.255480,0.272993,0.587630,0.643421,0.565574,0.590752,-2.979584,-3.000000
297,3.711763,3.270079,0.939133,0.740815,0.285663,0.400951,0.156354,0.213085,-1.589349,-1.673057
298,4.227479,4.421908,0.385177,0.425590,0.739687,0.677789,0.036994,0.041404,-1.335763,-1.325163
